## WhatsApp messages bulk sending
 
* **Autor:** Braulio Arteaga  
* **Fecha de creación:** 28-01-22  
* **Fecha de actualización:**  22-12-22

Este notebook tomo como referencia principal el repositorio [whatsapp-bulk-messages-without-saving-contacts](https://github.com/inforkgodara/whatsapp-bulk-messages-without-saving-contacts) de [Ramesh Kumar](https://github.com/inforkgodara).  

¡Gracias por tu contribución!

### 1. Requisitos: 


 * Instalar los siguinetes paquetes:
`!pip install pandas`  (data manipulation)  
`!pip install openpyxl`  
`!pip install selenium`  (scraping)    
`!pip install webdriver_manager`  (scraping)  
`!pip install datetime` (current date)  
`!pip install pytz` (time zone)

In [1]:
#!pip install openpyxl
#!pip install selenium 
#!pip install webdriver_manager  
#!pip install datetime 
#!pip install pytz 

### 2. Código:

In [1]:
# Importar packages
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time
from time import sleep
import pytz
from datetime import datetime

In [2]:
# Directorio actual de trabajo
import os
os.getcwd()

'C:\\Users\\braul\\Desktop\\02_projects\\01_GITHUB\\WhatsApp-messages-bulk-sending\\02_codes'

In [3]:
# Estableciento rutas relativas de trabajo (modificar)
MAIN = "C:/Users/braul/Desktop/02_projects/01_GITHUB/WhatsApp-messages-bulk-sending"
INPUT = MAIN + "/01_input"
OUTPUT = MAIN + "/03_output"

In [4]:
# Importación de la hoja excel (modificar)
excel_data = pd.read_excel(INPUT +'/list_phone_sms.xlsx')
data = pd.DataFrame(excel_data, columns = ['phone','message'])
data

,phone,message
0,9XXXXXXXX,¡Hola Juan! El siguiente mensaje es para recor...
1,962607427,¡Hola! Pedro! El siguiente mensaje es para rec...
2,9XXXXXXXZ,¡Hola Ronny! El siguiente mensaje es para reco...
3,9XXXXXXXW,¡Hola Dianela! El siguiente mensaje es para re...
4,9XXXXXXXV,¡Hola Antonella! El siguiente mensaje es para ...


In [5]:
# Añadir el código de país. Ejemplo: 51 = Perú (modificar):
data['phone'] = data['phone'].astype(str)
data["phone_cc"] = "51" + data["phone"] 
data

,phone,message,phone_cc
0,9XXXXXXXX,¡Hola Juan! El siguiente mensaje es para recor...,519XXXXXXXX
1,962607427,¡Hola! Pedro! El siguiente mensaje es para rec...,51962607427
2,9XXXXXXXZ,¡Hola Ronny! El siguiente mensaje es para reco...,519XXXXXXXZ
3,9XXXXXXXW,¡Hola Dianela! El siguiente mensaje es para re...,519XXXXXXXW
4,9XXXXXXXV,¡Hola Antonella! El siguiente mensaje es para ...,519XXXXXXXV


In [6]:
# Crear la estructura del reporte final (modificar):
report_send = pd.DataFrame(columns = ['phone_cc','message','status', 'motive'], index = list(range(len(data))))
report_send

,phone_cc,message,status,motive
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [7]:
############
# ENVÍO DE MENSAJES DE WSP (modificar)
############

## PASO 1: Iniciar sesión en tu WhatsApp Web 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://web.whatsapp.com')
input("Presione ENTER luego de iniciar sesión en Whatsapp Web. Posteriormente iniciará el envío de mensajes\n")


start = time.time()
exito = 0

for i in range(len(data)):
    try:
## PASO 2: Abrir el url correspondiente al wsp del individuo 
        report_send['phone_cc'][i] = data['phone_cc'][i] #reporte
        report_send['message'][i] = data['message'][i] #report
        url = 'https://web.whatsapp.com/send?phone=' + str(data['phone_cc'][i]) + '&text=' + data['message'][i]
        sent = False
        driver.get(url)
## PASO 3: Encontrar el elemento clickeable necesario (HTML) para enviar el sms predefinido
        try:
            click_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'tvf2evcx.oq44ahr5.lb5m6g5c.svlsagor.p2rjqpw5.epia9gcq'))) ##before: <button class="_4sWnG">
                
        except Exception as e:
            status1 = 'MENSAJE NO ENVIADO A ' + str(data['phone_cc'][i]) 
            motive1 = 'Número sin WhatsApp o problemas de internet'
            report_send['status'][i] = status1 #reporte 
            report_send['motive'][i] = motive1 #reporte 
            print(status1 + '. Motivo: ' + motive1)
        else:
## PASO 4: Click al elemento HTML identificado para enviar el sms predefinido
            sleep(2)
            click_btn.click()
            sent = True
            sleep(4)
            status2 = 'MENSAJE ENVIADO A ' + str(data['phone_cc'][i])
            report_send['status'][i] = status2 #reporte
            report_send['motive'][i] = "" #reporte
            exito += 1 
            print(status2)
    except Exception as e:
        status3 = "MENSAJE NO ENVIADO A " + str(data['phone_cc'][i])  
        motive2 = str(e)[9:34]
        report_send['status'][i] = status3 #reporte
        report_send['motive'][i] = motive2 #reporte
        print(status3 + '. Motivo: '+ motive2)
        
driver.quit() #Cerrar driver luego de finalizado el loop
print("\nEl loop terminó de ejecutarse.")


end = time.time() #Time of loop
print(f'\nSe enviaron {exito} mensajes con éxito y {len(data)-exito} tuvieron problemas')
print(f"\nLa ejecución del loop demoró {round((end-start)/60,2)} minutos") #Tiempo de duración del loop


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:01<00:00, 3.48MB/s]


Presione ENTER luego de iniciar sesión en Whatsapp Web. Posteriormente iniciará el envío de mensajes

MENSAJE NO ENVIADO A 519XXXXXXXX. Motivo: Número sin WhatsApp o problemas de internet
MENSAJE ENVIADO A 51962607427
MENSAJE NO ENVIADO A 519XXXXXXXZ. Motivo: Número sin WhatsApp o problemas de internet
MENSAJE NO ENVIADO A 519XXXXXXXW. Motivo: Número sin WhatsApp o problemas de internet
MENSAJE NO ENVIADO A 519XXXXXXXV. Motivo: Número sin WhatsApp o problemas de internet

El loop terminó de ejecutarse.

Se enviaron 1 mensajes con éxito y 4 tuvieron problemas

La ejecución del loop demoró 1.02 minutos


In [8]:
# Obtener fecha actual de Perú (modificar)
country_time = datetime.now(pytz.timezone('America/Lima')) #para averiguar tu zona horaria: print(pytz.all_timezones)
date = country_time.strftime("%d-%m-%y")

In [9]:
# Exportar reporte final
report_send.to_excel(OUTPUT + '/report_send_wsp_' + date + ".xlsx", index=False)

In [10]:
################
# REENVÍO DE MENSAJES NO ENVIADOS (código OPCIONAL)
################

# PASO 1: Obtener la "data" de los números que no recibieron el mensaje con éxito
report_not_send = report_send[report_send.status.str.contains('MENSAJE NO ENVIADO', regex = True, na = False)]
report_not_send = report_not_send.reset_index()
data = report_not_send[['phone_cc','message']]
data

# PASO 2: Ejecutar los bloques de código que realizan lo siguiente: 
## 1- "Crear la estructura del reporte final" (línea 6)
## 2- "ENVÍO DE SMS DE WSP" (línea 7)

,phone_cc,message
0,519XXXXXXXX,¡Hola Juan! El siguiente mensaje es para recor...
1,519XXXXXXXZ,¡Hola Ronny! El siguiente mensaje es para reco...
2,519XXXXXXXW,¡Hola Dianela! El siguiente mensaje es para re...
3,519XXXXXXXV,¡Hola Antonella! El siguiente mensaje es para ...
